**Module-1: In This Module We Import Dependencies.**

In [ ]:
!pip install mtcnn
!pip install matplotlib
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
import cv2
from PIL import Image 
from numpy import savez_compressed
from numpy import asarray
from os import listdir
from numpy import asarray
import csv
from time import strftime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Module-2: This Module Will Take Image Of Whole Class As Input And Give Single Faces As Output And Store It In Some Dir.(This Modeule Use MTCNN Model
)**

In [ ]:

img_crop = []
def save_faces(filename, result_list):
	# load the image
	data = pyplot.imread(filename)
	# plot each face as a subplot
	for i in range(len(result_list)):
		# get coordinates
		x1, y1, width, height = result_list[i]['box']
		x2, y2 = x1 + width, y1 + height
		cv2.imwrite(r"/content/drive/MyDrive/Facenet/cropped/I_{}.png".format(i),data[y1:y2, x1:x2])
print("saved")
 
filename = '/content/drive/MyDrive/Facenet/combined/2.jpg'
# load image from file
pixels = pyplot.imread(filename)
# create the detector, using default weights
detector = MTCNN()
# detect faces in the image
faces = detector.detect_faces(pixels)
# display faces on the original image
save_faces(filename, faces)

saved


FileNotFoundError: ignored

**Module-3: This Module Is For Removing Face Coordinates from Input Training DataSet.We will load DataSet Of Each Student For Training And Save All Coordinates in compressed file NewDataSet.npz.**

In [ ]:
def extract_image(image):
  store_face=[]
  img1 = Image.open(image)            
  img1 = img1.convert('RGB')          
  pixels = asarray(img1)              
  detector = MTCNN()                  
  f = detector.detect_faces(pixels)
  if f != []:
   x1,y1,w,h = f[0]['box']             
   x1, y1 = abs(x1), abs(y1)
   x2 = abs(x1+w)
   y2 = abs(y1+h)
  #locate the co-ordinates of face in the image
   store_face = pixels[y1:y2,x1:x2]
  # plt.imshow(store_face)
  image1 = Image.fromarray(store_face,'RGB')    #convert the numpy array to object
  image1 = image1.resize((160,160))             #resize the image
  face_array = asarray(image1)                  #image to array
  return face_array

def load_faces(directory):
  face = []
  i=1
  for filename in listdir(directory):
    path = directory + filename
    faces = extract_image(path)
    face.append(faces)
  return face

def load_dataset(directory):
  x, y = [],[]
  i=1
  for subdir in listdir(directory):
    path = directory + subdir + '/'
    #load all faces in subdirectory
    faces = load_faces(path)
    #create labels
    labels = [subdir for _ in range(len(faces))]
    # print(labels)
    #summarize
    print("%d There are %d images in the class %s:"%(i,len(faces),subdir))
    x.extend(faces)
    y.extend(labels)
    i=i+1
  return asarray(x),asarray(y)  

print(listdir('/content/drive/MyDrive/Facenet/training/'))
#load the datasets
trainX,trainY = load_dataset('/content/drive/MyDrive/Facenet/training/')
print(trainX.shape,trainY.shape)
# print(trainY)
#compress the data
savez_compressed('/content/drive/MyDrive/Facenet/DataSet.npz',trainX,trainY)

['user', 'Shyam', 'Sidharth', 'Udit', 'shubham', 'Vimal', 'Tilak', 'Vatsal', 'Smith', 'Tarik', 'Om', 'dhruv', 'Fenil', 'Harshad', 'Harsh', 'HarshBhalala', 'Kashyap', 'Kirtan', 'Manish', 'Shreyash', 'Bhumit', 'Darsh', 'Bhikho', 'Arpit']
1 There are 100 images in the class user:
2 There are 100 images in the class Shyam:
3 There are 50 images in the class Sidharth:
4 There are 100 images in the class Udit:
5 There are 100 images in the class shubham:
6 There are 100 images in the class Vimal:
7 There are 100 images in the class Tilak:
8 There are 100 images in the class Vatsal:
9 There are 100 images in the class Smith:
10 There are 95 images in the class Tarik:
11 There are 100 images in the class Om:
12 There are 100 images in the class dhruv:
13 There are 100 images in the class Fenil:
14 There are 100 images in the class Harshad:
15 There are 100 images in the class Harsh:
16 There are 100 images in the class HarshBhalala:
17 There are 100 images in the class Kashyap:
18 There are 10

**Module-4: In This Module We Will Load Our Pretrained Model Named FaceNet And Then By Loading NewDataSet.npz Obtained From Modeule-3 We Will Extract Embeddings(It is an list of 128 features of face of student) Of Each Training Face And Store It In Compressed File Named NewEmbeddings.npz**

In [ ]:
from numpy import load
from numpy import asarray
from numpy import expand_dims
from numpy import savez_compressed
from numpy import reshape
from keras.models import load_model 

#Generalize the data and extract the embeddings
def extract_embeddings(model,face_pixels):
  face_pixels = face_pixels.astype('float32')  #convert the entire data to float32(base)
  mean = face_pixels.mean()                    #evaluate the mean of the data
  std  = face_pixels.std()                     #evaluate the standard deviation of the data
  face_pixels = (face_pixels - mean)/std       
  samples = expand_dims(face_pixels,axis=0)    #expand the dimension of data 
  yhat = model.predict(samples)
  return yhat[0]

#load the compressed dataset and facenet keras model
data = load('/content/drive/MyDrive/FaceNet/DataSet.npz')
trainx, trainy = data['arr_0'],data['arr_1']
# print(trainy)
print(trainx.shape, trainy.shape)
model = load_model('/content/drive/MyDrive/FaceNet/facenet_keras.h5',compile=False)

#get the face embeddings
new_trainx = list()
for train_pixels in trainx:
  embeddings = extract_embeddings(model,train_pixels)
  new_trainx.append(embeddings)
new_trainx = asarray(new_trainx)             #convert the embeddings into numpy array
print(new_trainx.shape)

#compress the 128 embeddings of each face 
savez_compressed('/content/drive/MyDrive/FaceNet/Embeddings.npz',new_trainx,trainy)

(2675, 160, 160, 3) (2675,)
(2675, 128)


**Module-5: This Modeule Will Combine Old Embeddings(Embeddings.npz) With Newly Created Embeddings(NewEmbeddings.npz) Obtained From Module-4 and save it Which Furthure Used At Time Of Testing.**

In [ ]:
#For Combinning DataSet
import numpy as np
data_1 = np.load('/content/drive/MyDrive/FaceNet/DataSet.npz')
data_2 = np.load('/content/drive/MyDrive/FaceNet/NewDataSet.npz')
arr_0 = np.concatenate([data_1['arr_0'], data_2['arr_0']])
arr_1 = np.concatenate([data_1['arr_1'], data_2['arr_1']])
np.savez('/content/drive/MyDrive/FaceNet/FinalDataSet.npz', arr_0, arr_1)

#For Combinning Embeddings
data_1 = np.load('/content/drive/MyDrive/FaceNet/embeddings.npz')
data_2 = np.load('/content/drive/MyDrive/FaceNet/Newembeddings.npz')
arr_0 = np.concatenate([data_1['arr_0'], data_2['arr_0']])
arr_1 = np.concatenate([data_1['arr_1'], data_2['arr_1']])
np.savez('/content/drive/MyDrive/FaceNet/Finalembeddings.npz', arr_0, arr_1)
print("saved")

In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

**Module-6: This Modeule Is Final Modeule Use For Marking Attendance. In This Modeule We Will Load Coordinates File(DataSet.npz) And Embeddings File(Embeddings.npz). We Also Load Croped Images Obtained from Module-1 One By One And Compare Its Embeddings With Already Saved Embeddings Using SVM Classifier And Give Output As Label Of Matched Embedding.This Label Will Be the Name Of Student.**

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
from numpy import asarray
from numpy import array
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from numpy import expand_dims
from numpy import reshape
from numpy import load
from numpy import max
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import os

#extracting embeddings
def extract_embeddings(model,face_pixels):
  face_pixels = face_pixels.astype('float32')
  mean = face_pixels.mean()
  std  = face_pixels.std()
  face_pixels = (face_pixels - mean)/std
  samples = expand_dims(face_pixels,axis=0)
  yhat = model.predict(samples)
  return yhat[0]
directory='/content/drive/MyDrive/FaceNet/cropped/'
print(listdir('/content/drive/MyDrive/FaceNet/cropped/'))
dirs = os.listdir( directory )
model = load_model('/content/drive/MyDrive/FaceNet/facenet_keras.h5',compile=False)
data1 = load('/content/drive/MyDrive/FaceNet/DataSet.npz')
train_x,train_y = data1['arr_0'],data1['arr_1']

data = load('/content/drive/MyDrive/FaceNet/Embeddings.npz')
trainx,trainy= data['arr_0'],data['arr_1']

# i=0
name=[[]]
# data rows of csv file


for filename in dirs:
    Img = directory + filename

#load data and reshape the image
    img1 = Image.open(Img)            
    img1 = img1.convert('RGB')  
    img1 = img1.resize((160,160))        
    pixels = asarray(img1)    
    testx = pixels.reshape(-1,160,160,3)
    # print("Input test data shape: ",testx.shape)

    #find embeddings
    new_testx = list()
    for test_pixels in testx:
      embeddings = extract_embeddings(model,test_pixels)
      new_testx.append(embeddings)

    new_testx = asarray(new_testx)  
    # print("Input test embedding shape: ",new_testx.shape)
    # print(trainy.shape[0])
    # print("Loaded data: Train=%d , Test=%d"%(trainx.shape[0],new_testx.shape[0]))

    #normalize the input data
    in_encode = Normalizer(norm='l2')
    trainx = in_encode.transform(trainx)
    new_testx = in_encode.transform(new_testx)

    #create a label vector
    new_testy = trainy 
    out_encode = LabelEncoder()
    out_encode.fit(trainy)
    trainy = out_encode.transform(trainy)
    new_testy = out_encode.transform(new_testy)
    
    #define svm classifier model 
    model1 =SVC(kernel='linear', probability=True)
    model1.fit(trainx,trainy)

    #predict
    predict_train = model1.predict(trainx)
    predict_test = model1.predict(new_testx)


    #get the confidence score
    probability = model1.predict_proba(new_testx)
    confidence = max(probability)
    # print(confidence)

    if confidence > 0.45:

      acc_train = accuracy_score(trainy,predict_train)

      #display
      trainy_list = list(trainy)
      p=int(predict_test)
      if p in trainy_list:
        val = trainy_list.index(p)
        trainy = out_encode.inverse_transform(trainy)
        string = strftime('%H:%M:%S: %p')
        name.append([trainy[val],string])
        # print(trainy[val])
        # i+=1
      print(trainy[val])
    else:
      print("Unknown Face")
if len(name) > 0:
 header = ['Name','Time']
 with open('Attendance', 'w') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    for w in name:
      writer. writerow(w)
    

['Img_2.png', 'Img_3.png', 'Img_4.png', 'Img_5.png', 'Im_0.png', 'Im_1.png', 'Im_3.png', 'Im_4.png', 'Im_5.png', 'Im_6.png', 'Im_8.png', 'Im_9.png', 'Im_10.png', 'I_2.png', 'I_6.png']
Harshad
Bhikho
Harsh
Kashyap
Sidharth
Bhavya
Om
Manish
Tarik
HarshBhalala
Vatsal
Fenil
Smith
Unknown Face
Unknown Face


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
mv /content/drive/MyDrive/Facenet /content/

'/content'

**Module-7: This Module Will Be Used To Create Data Set.It Will Capture 100 Images Of Each Person Automatically And Store It At Specified Location Which Than Furthure Be Used For Training.**

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_default.xml")
def face_cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    for(x, y, w, h) in faces:
        face_cropped = img[y:y+h, x:x+w]
        return face_cropped



img_id = 0
cap=cv2.VideoCapture(0)
while True:
    ret,img=cap.read()
    cv2_imshow(img)

    if face_cropped(img) is not None:
        img_id += 1
        face = cv2.resize(face_cropped(img), (450, 450))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        dir="/content/drive/MyDrive/FaceNet/train/kashyap/"
        file_name_path=dir + str(img_id) + ".jpg"
        print(file_name_path)   
        cv2.imwrite(file_name_path, face)
        cv2.putText(face, str(img_id), (50, 50),cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)
        cv2.imshow("Cropped Face", face)

        if cv2.waitKey(1) == 13 or int(img_id) == 100:  
            break
cv2.destroyAllWindows()